# Part 1

## Load data from Marketing Input and Finance Input xlsx, identify sheet in the load command

In [1]:
import numpy as np
from openpyxl import load_workbook
import pandas as pd
pd.options.mode.chained_assignment = None
from datetime import datetime
fin = pd.read_excel('Finance Input.xlsx')
mar = pd.read_excel('Marketing Input.xlsx')

In [2]:
def write_excel(filename,sheetname,dataframe):
    book = load_workbook(filename)
    writer = pd.ExcelWriter(filename, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    dataframe.to_excel(writer, sheetname,index = False)
    writer.save()

In [3]:
def mdy_to_ymd(d):
    return datetime.strptime(d, '%b-%Y').strftime('%Y-%m-%d')

### Data Cleaning

In [4]:
fin[0] = 'Finance' 

In [5]:
fin = fin.rename(columns={0: 'Department'})

In [6]:
fin.columns == mar.columns # making sure that columns are same

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

## Append data from both files into a new dataframe

In [7]:
df = pd.concat([fin,mar])

## Unpivot the month columns from Jan 21 - Dec 22, so that months will show in column Month, and values in column "Value"

In [8]:
df_unpivoted = df.melt(id_vars=df.columns[:12], var_name='Month', value_name='Value')

## The new data set should have columns: Department, Segment, Brand, Market, GLName, Notes, Month, Value (remove 1 2 3 4 5 6 7)

In [9]:
col = df.columns[6:12]
clean = df_unpivoted.drop(columns = col)

In [10]:
clean.columns

Index(['Department', 'Segment', 'Brand', 'Market', 'GLName', 'Notes', 'Month',
       'Value'],
      dtype='object')

In [11]:
df_unpivoted.columns

Index(['Department', 'Segment', 'Brand', 'Market', 'GLName', 'Notes', '1', '2',
       '3', '5', '6', '7', 'Month', 'Value'],
      dtype='object')

## save new dataframe into a new xlsx file called Summary.xlsx

In [12]:
clean.to_excel("Summary.xlsx",index=False)

# Part 2

## Load data from the new Summary.xlsx file

In [13]:
new = pd.read_excel('Summary.xlsx')
new.head()

,Department,Segment,Brand,Market,GLName,Notes,Month,Value
0,Finance,B2C,A,NaN,Audit Fee,NaN,Jan-21,-1244.040700
1,Finance,B2C,A,NaN,Bank Fees,NaN,Jan-21,-88.459408
2,Finance,B2C,A,NaN,Business support solutions,NaN,Jan-21,-159.225500
3,Finance,B2C,A,NaN,Interest Expense,NaN,Jan-21,-9.622692
4,Finance,B2C,A,NaN,Project ACES,NaN,Jan-21,0.000000


## Split data into 2 dataframes based on the department column. Each department should have its own dataframe

In [14]:
finance = new.query("Department == 'Finance' ")
finance.Month = pd.to_datetime(finance.Month.apply(lambda x: x[:-2]+'20'+x[-2:]).apply(mdy_to_ymd))
finance.head()

,Department,Segment,Brand,Market,GLName,Notes,Month,Value
0,Finance,B2C,A,NaN,Audit Fee,NaN,2021-01-01,-1244.040700
1,Finance,B2C,A,NaN,Bank Fees,NaN,2021-01-01,-88.459408
2,Finance,B2C,A,NaN,Business support solutions,NaN,2021-01-01,-159.225500
3,Finance,B2C,A,NaN,Interest Expense,NaN,2021-01-01,-9.622692
4,Finance,B2C,A,NaN,Project ACES,NaN,2021-01-01,0.000000


In [15]:
marketing = new.query("Department == 'Marketing' ")
marketing.Month = pd.to_datetime(marketing.Month.apply(lambda x: x[:-2]+'20'+x[-2:]).apply(mdy_to_ymd))
marketing.head()

,Department,Segment,Brand,Market,GLName,Notes,Month,Value
37,Marketing,B2C,A,NaN,Audit Fee,NaN,2021-01-01,-1752.1700
38,Marketing,B2C,A,NaN,Bank Fees,NaN,2021-01-01,-106.5776
39,Marketing,B2C,A,NaN,Business support solutions,NaN,2021-01-01,-164.1500
40,Marketing,B2C,A,NaN,Interest Expense,NaN,2021-01-01,-320.7564
41,Marketing,B2C,A,NaN,Project ACES,NaN,2021-01-01,0.0000


## Update the Marketing output.xlsx(Data sheet) with the new data. Append below the existing data, leave data before

In [16]:
marketing2 = pd.read_excel('Marketing Output.xlsx',sheet_name = 'Data')
marketing2.head()

,Department,Segment,Brand,Market,GLName,Notes,Month,Value
0,Marketing,B2C,A,NaN,Audit Fee,NaN,2020-01-01,46
1,Marketing,B2C,A,NaN,Bank Fees,NaN,2020-02-01,896
2,Marketing,B2C,A,NaN,Business support solutions,NaN,2020-03-01,412
3,Marketing,B2C,A,NaN,Interest Expense,NaN,2020-04-01,208
4,Marketing,B2C,A,NaN,Project ACES,NaN,2020-05-01,728


In [17]:
marketing_new = pd.concat([marketing2,marketing])
marketing_new.head()

,Department,Segment,Brand,Market,GLName,Notes,Month,Value
0,Marketing,B2C,A,NaN,Audit Fee,NaN,2020-01-01,46.0
1,Marketing,B2C,A,NaN,Bank Fees,NaN,2020-02-01,896.0
2,Marketing,B2C,A,NaN,Business support solutions,NaN,2020-03-01,412.0
3,Marketing,B2C,A,NaN,Interest Expense,NaN,2020-04-01,208.0
4,Marketing,B2C,A,NaN,Project ACES,NaN,2020-05-01,728.0


In [18]:
write_excel('Marketing Output.xlsx','Data',marketing_new)

## Update the Finance output.xlsx(Data sheet) with the new data. Append below the existing data, leave data before

In [19]:
finance2 = pd.read_excel('Finance Output.xlsx',sheet_name = 'Data')

In [20]:
finance_new = pd.concat([finance2,finance])
finance_new.head()

,Department,Segment,Brand,Market,GLName,Notes,Month,Value
0,Finance,B2C,A,NaN,Audit Fee,NaN,2020-01-01,61.0
1,Finance,B2C,A,NaN,Bank Fees,NaN,2020-02-01,477.0
2,Finance,B2C,A,NaN,Business support solutions,NaN,2020-03-01,273.0
3,Finance,B2C,A,NaN,Interest Expense,NaN,2020-04-01,718.0
4,Finance,B2C,A,NaN,Project ACES,NaN,2020-05-01,750.0


In [21]:
write_excel('Finance Output.xlsx','Data',finance_new)